# **Processo de Modelagem de Dados**  

## **Objetivo: Identificação de Entidades, Atributos e Relacionamentos**  
Estruturar um modelo conceitual de banco de dados para um sistema de gestão inteligente de busca de vagas, com foco em automação, dados estruturados e análise de processos.  

---

## **1.1.Minimundo**  
### **Escopo do Sistema**  
O sistema é projetado para gerenciar a busca e aplicação a vagas de forma automatizada e baseada em dados. O candidato (único usuário inicial) interage com o sistema para:  
- Armazenar e versionar currículos e dados profissionais.  
- Buscar e aplicar a vagas em *batches* (lotes) periódicos.  
- Rastrear estágios do processo seletivo.  
- Analisar compatibilidade entre perfil e vagas.  

**Elementos irrelevantes**:  
- Contatos diretos de recrutadores (devido à alta rotatividade e baixo feedback).  
- Dados anteriores ao ensino superior (a menos que relevantes para a vaga).  

---

## **1.2.Entidades, Atributos e Relacionamentos**  

#### **1. CANDIDATO**  
**Descrição**: Representa o usuário do sistema (você) e seus dados profissionais.  
**Atributos**:  
- `id` (PK): Identificador único.  
- `name`: Nome completo.  
- `hard_skills`: Lista padronizada (ex: "Figma", "React") com níveis de proficiência.  
- `soft_skills`: Lista padronizada (ex: "Comunicação", "Liderança").  
- `work_experience`: Histórico profissional (cargo, empresa, período, conquistas).  
- `education`: Formação acadêmica.  
- `desired_salary_min` e `desired_salary_max`: Faixa salarial desejada.  
- `github_cv_url`: Link para currículo versionado no GitHub (JSON/YAML).  
- `preferred_work_model` (REMOTO, HÍBRIDO, PRESENCIAL).  
- `preferred_agreement_type` (CLT, PJ, etc.).  

**Comportamento do Sistema**:  
- Aplicações em *batches* (lotes) identificados por `batch_id` (ex: `v24s0110` = ano 2024, semana 01, década 10).  
- Logs de interações com IA (ex: simulações de entrevista).  

---

#### **2. VAGA**  
**Descrição**: Vagas de emprego buscadas ou aplicadas.  
**Atributos**:  
- `id` (PK): Identificador único.  
- `title`: Título padronizado no formato `[NÍVEL][PAPEL][SENIORIDADE][ESPECIALIDADE][CONTEXTO]` (ex: "PLENO|PRODUCT_DESIGNER|SENIOR|DESIGN_SYSTEM|E-COMMERCE").  
- `company_id` (FK): Referência à empresa.  
- `salary_min` e `salary_max`: Faixa salarial (baseada em Glassdoor ou relatos).  
- `requirements`:  
  - `hard_skills_required`: Lista padronizada.  
  - `soft_skills_required`: Lista padronizada.  
  - `nice_to_have`: Habilidades desejáveis.  
- `keywords`: Palavras-chave para busca (ex: "DESIGN_SYSTEM AND E-COMMERCE").  
- `benefits`: Classificação (flexibilidade, saúde, bônus).  
- `work_model` (REMOTO, HÍBRIDO, PRESENCIAL).  
- `agreement_type` (CLT, PJ, etc.).  
- `status`: Estágio atual (ex: "SCREENING", "ENTREVISTA_TÉCNICA", "OFFER").  
- `application_date`: Data da aplicação.  
- `batch_id`: Lote associado à aplicação.  

**Rastreamento**:  
- Histórico de mudanças de status (`status_logs`).  

---

#### **3. EMPRESA**  
**Descrição**: Organizações que oferecem vagas.  
**Atributos**:  
- `id` (PK): Identificador único.  
- `name`: Nome da empresa.  
- `sector` (TECH, FINANCEIRO, SAÚDE).  
- `size` (STARTUP, MULTINACIONAL).  
- `employee_count_range` (ex: "11-20").  
- `glassdoor_rating`: Nota no Glassdoor.  
- `awards`: Prêmios (ex: "GPTW", "FORBES").  

**Descartado**:  
- Contatos de recrutadores (foco em automação).  

---

#### **4. PROCESSO_SELETIVO** *(Opcional para IA)*  
**Descrição**: Etapas automatizadas de seleção.  
**Atributos**:  
- `id` (PK): Identificador único.  
- `vaga_id` (FK): Vaga associada.  
- `current_stage`: Estágio atual (ex: "CHATBOT_SCREENING").  
- `automated_interviews`: Logs de entrevistas simuladas por IA.  

---

## **1.3.Modelagem Conceitual**  
```mermaid
erDiagram
    CANDIDATO ||--o{ VAGA : "aplica"
    CANDIDATO {
        int id PK
        string name
        string[] hard_skills
        string[] soft_skills
        json work_experience
        json education
        decimal desired_salary_min
        decimal desired_salary_max
        string github_cv_url
        string preferred_work_model
        string preferred_agreement_type
    }
    VAGA ||--o{ EMPRESA : "pertence"
    VAGA {
        int id PK
        int company_id FK
        string title
        decimal salary_min
        decimal salary_max
        json requirements
        string[] keywords
        json benefits
        string work_model
        string agreement_type
        string status
        date application_date
        string batch_id
    }
    EMPRESA {
        int id PK
        string name
        string sector
        string size
        string employee_count_range
        float glassdoor_rating
        string[] awards
    }
    PROCESSO_SELETIVO }|--|| VAGA : "referencia"
    PROCESSO_SELETIVO {
        int id PK
        int vaga_id FK
        string current_stage
        json automated_interviews
    }
```

---

## **1.4.Exemplo de Fluxo**  
1. **Candidato**:  
   - Atualiza `github_cv_url` com currículo em YAML.  
   - Sistema inicia *batch* `v24s0110` (10 vagas/semana).  

2. **Vaga**:  
   - Padronizada como `"PLENO|UX_WRITER|SENIOR|TECH|SAAS"`.  
   - Salário estimado: `R$ 8.000-10.000` (fonte: Glassdoor).  
   - Status atual: `"ENTREVISTA_TÉCNICA"`.  

3. **Empresa**:  
   - Setor: `"TECH"`, premiação: `"GPTW 2023"`.  

### **2 - Processo de Modelagem de Dados: Levantamento de Requisitos**  

## **2.1.Levantamento de Requisitos**  
*(Foco em Web Scraping, Automação e Gestão de Dados Estruturados)*  

---

#### **1.Requisitos Funcionais**  

| ID     | Descrição                          | Detalhes Técnicos |  
|--------|------------------------------------|-------------------|  
| **RF01** | Padronização de campos de vagas   | - Formato: `[NÍVEL][PAPEL][SENIORIDADE][ESPECIALIDADE][CONTEXTO]`<br>- Validação via Regex (`/^[A-Z]+\|[A-Z_]+\|[A-Z]+\|[A-Z_]+\|[A-Z_]+$/`) |  
| **RF02** | Web Scraper multi-plataforma      | - Extração automática de:<br>  - `job_title` (com conversão para padrão RF01)<br>  - `salary_range` (Glassdoor/LinkedIn)<br>  - `requirements` (hard/soft skills)<br>  - `benefits`<br>- Suporte para LinkedIn, Gupy, Glassdoor |  
| **RF03** | Sistema de aplicações em *batch*  | - Configuração de limite (ex: 25 vagas/semana)<br>- Identificador único (`batch_id` = `vYYsWWDD`)<br>- Logs de aplicações (`status`, `timestamp`) |  
| **RF04** | Rastreamento de processos         | - Histórico de mudanças:<br>  - `status` (ex: "SCREENING", "TECH_INTERVIEW")<br>  - `change_reason` (feedback, timeout)<br>  - `events` (e-mails, testes) |  
| **RF05** | Análise de compatibilidade        | - Matching automático entre:<br>  - `candidate_skills` x `job_requirements`<br>  - `desired_salary` x `salary_range` |  

---

#### **2.Requisitos Não-Funcionais**  

| ID     | Descrição                          |  
|--------|------------------------------------|  
| **RNF01** | Resilência do Scraper             | - Rotação de User-Agents<br>- Delay aleatório (3-7s)<br>- Fallback para CAPTCHAs (Armazenar HTML bruto) |  
| **RNF02** | Versionamento de dados            | - Armazenamento do currículo em GitHub (YAML/JSON)<br>- Tagging por versão (`v1.0.0`) |  
| **RNF03** | Performance do Banco de Dados     | - Otimizado para:<br>  - Consultas booleanas (`keywords`)<br>  - Armazenamento de logs brutos (HTML scraped) |  
| **RNF04** | Segurança                         | - Criptografia de dados sensíveis (ex: salários)<br>- Autenticação via OAuth para GitHub |  

---

### **2.2.Regras de Negócio**  

| ID     | Regra                              | Lógica |  
|--------|------------------------------------|--------|  
| **RN01** | Filtro salarial automático        | - Alertar se `job_salary_max` < `candidate_salary_min * 0.8` |  
| **RN02** | Validação de vagas scraped        | - Campos obrigatórios:<br>  - `job_title` (100% preenchido)<br>  - `requirements` (mín. 70%)<br>  - `work_model` (REMOTO/HÍBRIDO/PRESENCIAL) |  
| **RN03** | Prevenção de duplicatas           | - Hash MD5 da URL como chave única |  
| **RN04** | Priorização de vagas              | - Score baseado em:<br>  - Match de skills (60%)<br>  - Faixa salarial (30%)<br>  - Reputação da empresa (10%) |  

---

### **2.3.Fluxo do Scraper (Pseudocódigo Atualizado)**  
```python  
def scrape_and_process_job(url):  
    try:  
        # Etapa 1: Extração  
        raw_html = scrape_with_selenium(url, user_agent=random_agent())  # RNF01  
        
        # Etapa 2: Padronização  
        job_data = {  
            'title': standardize_title(raw_html.title),  # RF01  
            'salary': extract_salary(raw_html),          # RF02  
            'requirements': parse_skills(raw_html)       # RF05  
        }  
        
        # Etapa 3: Validação  
        if validate_job(job_data):                      # RN02  
            job_data['url_hash'] = md5(url)             # RN03  
            save_to_database(job_data)                  # RNF03  
            log_status(url, "SUCCESS")  
        else:  
            log_status(url, "INVALID_DATA")  
            
    except Exception as e:  
        save_raw_html(url, raw_html)  # Fallback        # RNF01  
        log_status(url, f"ERROR: {str(e)}")  
```

---

## **3 - Processo de Modelagem de Dados: Identificação de Entidades**  

### **Identificação das Entidades**  
*(Foco em Automação e Rastreamento de Processos de Recrutamento)*  

---

#### **3.1 Classificação das Entidades**  

#### **Entidades Fortes (Independentes)**  
| Entidade          | Descrição                                                                 | Key Attributes (Exemplos)                                                                 |  
|-------------------|---------------------------------------------------------------------------|------------------------------------------------------------------------------------------|  
| **Candidato**     | Representa o usuário do sistema                                          | `id`, `technical_skills`, `salary_expectation`, `github_resume`, `current_batch`         |  
| **Empresa**       | Organizações que publicam vagas                                          | `id`, `name`, `sector`, `size`, `glassdoor_rating`                                      |  
| **Plataforma**    | Fontes de vagas (LinkedIn, Glassdoor, etc.)                             | `id`, `name`, `base_url`, `scraping_difficulty`                                         |  

#### **Entidades Fracas (Dependentes)**  
| Entidade               | Relacionamento                     | Key Attributes (Exemplos)                                                                 |  
|------------------------|------------------------------------|------------------------------------------------------------------------------------------|  
| **Vaga**               | Pertence a `Empresa` e `Plataforma` | `id`, `standardized_title`, `estimated_salary`, `original_url`, `current_status`, `batch_id` |  
| **ProcessoSeletivo**   | Associado a `Vaga`                | `id`, `current_stage`, `last_feedback`, `last_update_date`                              |  
| **ScrapingLog**        | Registro de operações do scraper  | `id`, `url`, `timestamp`, `status`, `raw_data` (JSON)                                   |  
| **Candidatura**        | Relaciona `Candidato` e `Vaga`    | `id`, `application_date`, `submission_method`, `compatibility_score`                    |  

---

#### **3.2 Relacionamentos Principais**  

1. **Empresa (1) → (N) Vaga**  
   - Uma empresa pode ter múltiplas vagas ativas.  
2. **Vaga (1) → (N) ProcessoSeletivo**  
   - Cada vaga possui um histórico de etapas do processo.  
3. **Plataforma (1) → (N) Vaga**  
   - Cada plataforma fonte contém diversas vagas.  
4. **Candidato (1) → (N) Candidatura**  
   - O candidato pode se aplicar a múltiplas vagas.  

---

#### **3.3 Exemplo de Estrutura em SQL**  

```sql  
-- Tabela Candidato (Entidade Forte)  
CREATE TABLE candidato (  
    id SERIAL PRIMARY KEY,  
    technical_skills JSONB NOT NULL,  -- Ex: {"design_system": "advanced", "figma": "expert"}  
    salary_expectation NUMERIC(10,2) CHECK (salary_expectation > 0),  
    github_resume VARCHAR(255) UNIQUE  
);  

-- Tabela Vaga (Entidade Fraca)  
CREATE TABLE vaga (  
    id SERIAL PRIMARY KEY,  
    standardized_title VARCHAR(100) NOT NULL,  -- Formato: "MID|UX_DESIGNER|JUNIOR|E-COMMERCE"  
    empresa_id INTEGER REFERENCES empresa(id),  
    plataforma_id INTEGER REFERENCES plataforma(id),  
    original_url VARCHAR(255) UNIQUE  
);  
```  

---

## **4 - Processo de Modelagem de Dados: Definição dos Relacionamentos**

### **Definição dos Relacionamentos**
*(Modelagem Completa com Cardinalidades e Entidades Associativas)*

---

#### **4.1 Relacionamentos Identificados**

| **Entidade A** | **Relacionamento** | **Entidade B** | **Cardinalidade** | **Detalhes** |
|----------------|--------------------|----------------|-------------------|--------------|
| **Candidato** | REALIZA | **Candidatura** | 1:N | Um candidato pode se candidatar a múltiplas vagas |
| **Vaga** | RECEBE | **Candidatura** | 1:N | Uma vaga pode receber múltiplas candidaturas |
| **Empresa** | PUBLICA | **Vaga** | 1:N | Uma empresa pode publicar múltiplas vagas |
| **Plataforma** | HOSPEDA | **Vaga** | 1:N | Uma plataforma pode listar múltiplas vagas |
| **Vaga** | POSSUI | **ProcessoSeletivo** | 1:N | Cada vaga tem um histórico de processos |
| **Vaga** | REGISTRA | **ScrapingLog** | 1:1 | Cada vaga coletada tem um registro único |

---

#### **4.2 Entidades Associativas**

##### **CompatibilidadeCandidatoVaga**
- **Atributos**:
  - `compatibility_score` (0-100%)
  - `evaluation_date`
  - `matching_algorithm_version`
- **Função**:
  - Armazena resultados do matching automático entre candidato e vaga

##### **EventoProcesso**
- **Atributos**:
  - `event_type` ("INTERVIEW", "TEST", "FEEDBACK")
  - `event_date`
  - `details` (JSON)
  - `participants` (array de IDs)
- **Função**:
  - Registra todos os eventos do processo seletivo

---

#### **4.3 Implementação em SQL**

```sql
-- Tabela de Candidatura (Relacionamento N:M entre Candidato e Vaga)
CREATE TABLE candidatura (
    id SERIAL PRIMARY KEY,
    candidate_id INTEGER NOT NULL REFERENCES candidato(id),
    job_id INTEGER NOT NULL REFERENCES vaga(id),
    application_date TIMESTAMPTZ DEFAULT NOW(),
    compatibility_score DECIMAL(5,2),
    application_method VARCHAR(20),
    UNIQUE (candidate_id, job_id)
);

-- Tabela de Processo Seletivo
CREATE TABLE processo_seletivo (
    id SERIAL PRIMARY KEY,
    job_id INTEGER NOT NULL REFERENCES vaga(id),
    current_stage VARCHAR(30) NOT NULL,
    last_update TIMESTAMPTZ DEFAULT NOW()
);

-- Tabela de Eventos do Processo
CREATE TABLE evento_processo (
    id SERIAL PRIMARY KEY,
    process_id INTEGER NOT NULL REFERENCES processo_seletivo(id),
    event_type VARCHAR(30) NOT NULL,
    event_data JSONB
);
```

---

#### **4.4 Diagrama Conceitual**

```mermaid
erDiagram
    CANDIDATO ||--o{ CANDIDATURA : "REALIZA"
    VAGA ||--o{ CANDIDATURA : "RECEBE"
    EMPRESA ||--o{ VAGA : "PUBLICA"
    PLATAFORMA ||--o{ VAGA : "HOSPEDA"
    VAGA ||--o{ PROCESSO_SELETIVO : "POSSUI"
    PROCESSO_SELETIVO ||--o{ EVENTO_PROCESSO : "REGISTRA"
    CANDIDATURA }|--|| COMPATIBILIDADE : "AVALIA"
```

---

## **5 - Processo de Modelagem de Dados: Especificação dos Atributos**

### **Especificação dos Atributos**
*(Classificação Completa por Entidade com Tipos e Restrições)*

---

#### **5.1 Entidade `CANDIDATO`**

| Atributo | Tipo | Descrição | Restrições |
|----------|------|-----------|------------|
| `id` | **Chave Primária** (SERIAL) | Identificador único | PRIMARY KEY |
| `technical_skills` | **Multivalorado** (JSONB) | Habilidades técnicas formatadas: `{"skill": "level"}` | NOT NULL |
| `salary_expectation` | **Composto** (JSONB) | Faixa salarial: `{"min": 5000, "max": 8000, "currency": "BRL"}` | CHECK (min < max) |
| `github_resume` | **Simples** (VARCHAR) | URL do currículo versionado | UNIQUE |
| `current_batch` | **Simples** (CHAR(8)) | Identificador do lote atual | Formato `vYYsWWDD` |

---

#### **5.2 Entidade `VAGA`**

| Atributo | Tipo | Descrição | Restrições |
|----------|------|-----------|------------|
| `id` | **Chave Primária** (SERIAL) | Identificador único | PRIMARY KEY |
| `standardized_title` | **Simples** (formato_cargo) | Título padronizado | Regex validation |
| `estimated_salary` | **Composto** (JSONB) | Faixa salarial estimada | CHECK (min < max) |
| `requirements` | **Multivalorado** (JSONB) | Requisitos da vaga | NOT NULL |
| `original_url` | **Simples** (VARCHAR) | URL original da vaga | UNIQUE |
| `current_status` | **Simples** (ENUM) | Status atual da vaga | DEFAULT 'ABERTA' |

---

#### **5.3 Entidade `PROCESSO_SELETIVO`**

| Atributo | Tipo | Descrição | Restrições |
|----------|------|-----------|------------|
| `id` | **Chave Primária** (SERIAL) | Identificador único | PRIMARY KEY |
| `current_stage` | **Simples** (ENUM) | Etapa atual do processo | NOT NULL |
| `event_history` | **Multivalorado** (JSONB[]) | Histórico de eventos | -- |

---

#### **5.4 Entidade Associativa `CANDIDATURA`**

| Atributo | Tipo | Descrição | Restrições |
|----------|------|-----------|------------|
| `id` | **Chave Primária** (SERIAL) | Identificador único | PRIMARY KEY |
| `compatibility_score` | **Simples** (DECIMAL) | Percentual de compatibilidade | CHECK (0 <= score <= 100) |
| `submission_method` | **Simples** (ENUM) | Método de envio | DEFAULT 'BATCH' |

---

#### **5.5 Tipos de Dados Especiais**

```sql
-- Domínio para título padronizado
CREATE DOMAIN formato_cargo AS VARCHAR(100)
CHECK (VALUE ~ '^[A-Z]+\|[A-Z_]+\|[A-Z]+\|[A-Z_]+\|[A-Z_]+$');

-- Tipo ENUM para status
CREATE TYPE status_vaga AS ENUM ('ABERTA', 'EM_ANDAMENTO', 'FECHADA');
```

---

## **6 - Processo de Modelagem de Dados: Validação com Regras de Negócio**

## **Validação e Integridade para Gestão de Vagas**
*(Implementação Completa de Restrições e Regras de Negócio)*

---

#### **6.1 Validação das Regras de Negócio**

| **Regra** | **Descrição** | **Implementação** | **Mecanismo** |
|-----------|--------------|------------------|---------------|
| **RN01** Filtro salarial | Vagas com salário 20% abaixo da pretensão do candidato são marcadas | Comparação entre `salary_expectation` (CANDIDATO) e `estimated_salary` (VAGA) | Trigger `validate_salary` |
| **RN02** Validação de vagas | Campos obrigatórios: título padronizado e requisitos mínimos | CHECK constraints em `VAGA` | `standardized_title IS NOT NULL AND requirements->'hard_skills' != '[]'` |
| **RN03** Prevenção de duplicatas | URL única para cada vaga | Constraint UNIQUE em `original_url` | Índice único + verificação no scraper |
| **RN04** Limite de batch | Controle de aplicações por lote | Contagem em `CANDIDATURA` por `batch_id` | Stored procedure `check_batch_limit` |

---

#### **6.2 Verificação de Integridade**

| **Relacionamento** | **Regra** | **Implementação** |
|--------------------|----------|------------------|
| CANDIDATURA → VAGA | Candidatura requer vaga existente | FOREIGN KEY com `ON DELETE CASCADE` |
| PROCESSO_SELETIVO → VAGA | Processo só para vagas ativas | FOREIGN KEY + CHECK (`current_status != 'FECHADA'`) |
| SCRAPING_LOG → VAGA | Logs podem existir sem vaga (para falhas) | FOREIGN KEY com `ON DELETE SET NULL` |

---

#### **6.3 Implementação em SQL**

##### **Trigger para Validação Salarial (RN01)**
```sql
CREATE OR REPLACE FUNCTION validate_salary()
RETURNS TRIGGER AS $$
BEGIN
    IF (NEW.estimated_salary->>'max')::NUMERIC < 
       (SELECT (salary_expectation->>'min')::NUMERIC * 0.8 
        FROM candidato 
        WHERE id = NEW.candidate_id) THEN
        NEW.compatibility_status = 'INCOMPATIBLE';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trg_validate_salary
BEFORE INSERT OR UPDATE ON candidatura
FOR EACH ROW EXECUTE FUNCTION validate_salary();
```

##### **Constraint para Vagas Válidas (RN02)**
```sql
ALTER TABLE vaga
ADD CONSTRAINT chk_valid_job CHECK (
    standardized_title IS NOT NULL AND
    jsonb_array_length(requirements->'hard_skills') > 0
);
```

---

#### **6.4 Casos de Teste**

| **Cenário** | **Entrada** | **Resultado Esperado** |
|-------------|------------|-----------------------|
| Salário incompatível | Vaga: max R$4.000 | Candidatura marcada como "INCOMPATIBLE" |
| Vaga sem hard skills | `requirements: {"hard_skills": []}` | Erro na inserção |
| URL duplicada | `original_url` existente | Violação de constraint UNIQUE |

---

## **7 - Documentação Final do Modelo de Dados para Gestão de Vagas**

### **Documentação e Ferramentas**

#### **7.1 Diagrama Entidade-Relacionamento (DER) Completo**

```mermaid
erDiagram
    CANDIDATO ||--o{ CANDIDATURA : "REALIZA"
    CANDIDATO {
        SERIAL id PK
        JSONB technical_skills
        NUMERIC[] salary_expectation
        VARCHAR github_resume
        CHAR(8) current_batch
    }
    
    VAGA ||--o{ CANDIDATURA : "RECEBE"
    VAGA {
        SERIAL id PK
        VARCHAR standardized_title
        JSONB estimated_salary
        JSONB requirements
        VARCHAR original_url
        ENUM current_status
    }
    
    EMPRESA ||--o{ VAGA : "PUBLICA"
    EMPRESA {
        SERIAL id PK
        VARCHAR name
        VARCHAR sector
        VARCHAR size
    }
    
    PLATAFORMA ||--o{ VAGA : "HOSPEDA"
    PLATAFORMA {
        SERIAL id PK
        VARCHAR name
        VARCHAR base_url
    }
    
    VAGA ||--o{ PROCESSO_SELETIVO : "POSSUI"
    PROCESSO_SELETIVO {
        SERIAL id PK
        ENUM current_stage
        JSONB[] event_history
    }
    
    VAGA ||--|| SCRAPING_LOG : "REGISTRA"
    SCRAPING_LOG {
        VARCHAR(64) url_hash PK
        JSONB raw_data
        VARCHAR status
        TIMESTAMPTZ timestamp
    }
```

#### **7.2 Ferramentas Recomendadas para Modelagem**

| Ferramenta | Tipo | Melhor Para | Link |
|------------|------|-------------|------|
| **dbdiagram.io** | Online | Sintaxe simples e exportação para SQL | [dbdiagram.io](https://dbdiagram.io) |
| **pgAdmin ERD** | Desktop | Integração com PostgreSQL | Incluído no pgAdmin |
| **Lucidchart** | Online | Colaboração em equipe | [lucid.app](https://lucid.app) |
| **DBeaver** | Desktop | Multiplataforma e open-source | [dbeaver.io](https://dbeaver.io) |

#### **7.3 Script SQL para Implementação**

```sql
-- Domínios customizados
CREATE DOMAIN job_title_format AS VARCHAR(100) 
CHECK (VALUE ~ '^[A-Z]+\|[A-Z_]+\|[A-Z]+\|[A-Z_]+\|[A-Z_]+$');

CREATE TYPE job_status AS ENUM ('OPEN', 'IN_PROGRESS', 'CLOSED');

-- Tabela Candidato
CREATE TABLE candidato (
    id SERIAL PRIMARY KEY,
    technical_skills JSONB NOT NULL,
    salary_expectation NUMERIC(10,2)[] CHECK (
        array_length(salary_expectation, 1) = 2 
        AND salary_expectation[1] < salary_expectation[2]
    ),
    github_resume VARCHAR(255) UNIQUE,
    current_batch CHAR(8) CHECK (current_batch ~ '^v\d{2}s\d{2}[1-3]\d$')
);

-- Tabela Vaga
CREATE TABLE vaga (
    id SERIAL PRIMARY KEY,
    standardized_title job_title_format NOT NULL,
    estimated_salary JSONB CHECK (
        (estimated_salary->>'min')::NUMERIC < (estimated_salary->>'max')::NUMERIC
    ),
    requirements JSONB NOT NULL CHECK (
        requirements ? 'hard_skills' AND 
        jsonb_array_length(requirements->'hard_skills') > 0
    ),
    original_url VARCHAR(255) UNIQUE,
    current_status job_status DEFAULT 'OPEN',
    empresa_id INTEGER REFERENCES empresa(id),
    plataforma_id INTEGER REFERENCES plataforma(id)
);
```

#### **7.4 Checklist de Validação**

1. [ ] Todos os relacionamentos estão mapeados
2. [ ] Atributos multivalorados estão como JSONB/arrays
3. [ ] Constraints de negócio implementadas
4. [ ] Índices criados para campos de busca frequente
5. [ ] Documentação completa gerada

#### **7.5 Próximos Passos**

1. **Implementar o modelo físico** no banco de dados PostgreSQL
2. **Configurar índices** para otimização
3. **Desenvolver stored procedures** para operações complexas
4. **Criar documentação técnica** detalhada


# **Modelo Lógico de Banco de Dados - Sistema de Recrutamento**  

## **1. Objetivo Geral**  
Transformar o modelo conceitual em estrutura lógica detalhada, definindo tabelas, colunas, relacionamentos, chaves e regras de integridade para implementação em SGBD relacional.  

---



## **2. Definição das Tabelas e Atributos**  

### **2.1. Tabela: CANDIDATE**  
| Atributo | Tipo | Descrição | Restrições |  
|----------|------|-----------|------------|  
| `id` | SERIAL | Identificador único | PRIMARY KEY |  
| `full_name` | VARCHAR(100) | Nome completo | NOT NULL |  
| `min_salary_expectation` | NUMERIC(10,2) | Expectativa salarial mínima | CHECK > 0 |  
| `max_salary_expectation` | NUMERIC(10,2) | Expectativa salarial máxima | CHECK > min_salary |  
| `github_cv_url` | VARCHAR(255) | URL do currículo no GitHub | UNIQUE |  
| `preferred_work_model` | VARCHAR(10) | Modelo de trabalho preferido | CHECK IN ('REMOTE','HYBRID','ON_SITE') |  
| `preferred_contract_type` | VARCHAR(10) | Tipo de contrato preferido | CHECK IN ('CLT','PJ','INTERN') |  

```mermaid
erDiagram
    CANDIDATE {
        int id PK
        varchar(100) full_name
        numeric min_salary_expectation
        numeric max_salary_expectation
        varchar(255) github_cv_url
        varchar preferred_work_model
        varchar preferred_contract_type
    }
```

### **2.2. Tabela: JOB**  
| Atributo | Tipo | Descrição | Restrições |  
|----------|------|-----------|------------|  
| `id` | SERIAL | Identificador único | PRIMARY KEY |  
| `standardized_title` | VARCHAR(150) | Título padronizado | Regex: `^[A-Z]+\|[A-Z_]+\|[A-Z]+\|[A-Z_]+\|[A-Z_]+$` |  
| `company_id` | INTEGER | Referência à empresa | FK, NOT NULL |  
| `min_salary` | NUMERIC(10,2) | Salário mínimo ofertado | CHECK >= 0 |  
| `max_salary` | NUMERIC(10,2) | Salário máximo ofertado | CHECK > min_salary |  
| `application_status` | VARCHAR(20) | Status da aplicação | CHECK IN ('SCREENING','TECH_INTERVIEW','OFFER') |  
| `batch_id` | VARCHAR(10) | Identificador do lote | Regex: `^v\d{2}s\d{2}[1-3]\d$` |  

```mermaid
erDiagram
    JOB {
        int id PK
        varchar(150) standardized_title
        int company_id FK
        numeric min_salary
        numeric max_salary
        varchar application_status
        varchar batch_id
    }
```

### **2.3. Tabela: COMPANY**  
| Atributo | Tipo | Descrição | Restrições |  
|----------|------|-----------|------------|  
| `id` | SERIAL | Identificador único | PRIMARY KEY |  
| `company_name` | VARCHAR(100) | Nome legal da empresa | NOT NULL |  
| `sector` | VARCHAR(20) | Setor de atuação | CHECK IN ('TECH','FINANCE','HEALTH') |  
| `glassdoor_rating` | FLOAT | Avaliação no Glassdoor | CHECK BETWEEN 0.0 AND 5.0 |  
| `employee_range` | VARCHAR(20) | Faixa de funcionários | - |  

```sql
CREATE TABLE company (
    id SERIAL PRIMARY KEY,
    company_name VARCHAR(100) NOT NULL,
    sector VARCHAR(20) CHECK (sector IN ('TECH','FINANCE','HEALTH')),
    glassdoor_rating FLOAT CHECK (glassdoor_rating BETWEEN 0.0 AND 5.0),
    employee_range VARCHAR(20)
);
```

### **2.4. Tabelas Normalizadas (1FN)**  
**CANDIDATE_SKILL**  
```mermaid
erDiagram
    CANDIDATE_SKILL {
        int candidate_id PK, FK
        varchar skill_type PK
        varchar skill_name PK
        varchar proficiency
    }
    CANDIDATE ||--o{ CANDIDATE_SKILL : "has"
```

**JOB_REQUIREMENT**  
```sql
CREATE TABLE job_requirement (
    job_id INTEGER NOT NULL REFERENCES job(id) ON DELETE CASCADE,
    requirement_type VARCHAR(20) NOT NULL CHECK (requirement_type IN ('MANDATORY','NICE_TO_HAVE')),
    description TEXT NOT NULL,
    PRIMARY KEY (job_id, requirement_type, description)
);
```

---

## **3. Relacionamentos e Integridade Referencial**  

### **3.1. Diagrama de Relacionamentos**  
```mermaid
erDiagram
    CANDIDATE ||--o{ APPLICATION : "submits"
    JOB ||--o{ APPLICATION : "receives"
    COMPANY ||--o{ JOB : "publishes"
    JOB ||--|| SELECTION_PROCESS : "has"
    JOB }|--|| PLATFORM : "sourced_from"
    CANDIDATE ||--o{ CANDIDATE_SKILL : "has"
    JOB ||--o{ JOB_REQUIREMENT : "requires"

    APPLICATION {
        int id PK
        int candidate_id FK
        int job_id FK
        timestamp application_date
    }
    SELECTION_PROCESS {
        int id PK
        int job_id FK
        varchar current_stage
    }
```

### **3.2. Regras de Integridade**  
| Relacionamento | Cardinalidade | Ação ON DELETE |  
|----------------|---------------|----------------|  
| JOB → COMPANY | N:1 | CASCADE |  
| APPLICATION → CANDIDATE | N:1 | RESTRICT |  
| APPLICATION → JOB | N:1 | RESTRICT |  
| SELECTION_PROCESS → JOB | 1:1 | CASCADE |  

---

## **4. Regras de Negócio Implementadas**  

### **4.1. Validação Salarial**  
```sql
-- Expectativas salariais coerentes
ALTER TABLE candidate ADD CONSTRAINT chk_salary_expectation 
CHECK (max_salary_expectation > min_salary_expectation);

-- Salários de vagas válidos
ALTER TABLE job ADD CONSTRAINT chk_job_salary 
CHECK (max_salary > min_salary);
```

### **4.2. Limite de Candidaturas por Batch**  
```sql
CREATE OR REPLACE FUNCTION check_batch_limit()
RETURNS TRIGGER AS $$
BEGIN
    IF (SELECT COUNT(*) FROM application 
        WHERE batch_id = NEW.batch_id) >= 20 
    THEN
        RAISE EXCEPTION 'Limite de 20 candidaturas por batch atingido';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trg_batch_limit
BEFORE INSERT ON application
FOR EACH ROW EXECUTE FUNCTION check_batch_limit();
```

### **4.3. Transições de Status Válidas**  
```sql
CREATE TABLE selection_process_status (
    job_id INTEGER PRIMARY KEY REFERENCES job(id) ON DELETE CASCADE,
    current_status VARCHAR(20) NOT NULL CHECK (current_status IN ('SCREENING','TECH_INTERVIEW','OFFER')),
    previous_status VARCHAR(20),
    CONSTRAINT valid_transition CHECK (
        (previous_status IS NULL AND current_status = 'SCREENING') OR
        (previous_status = 'SCREENING' AND current_status = 'TECH_INTERVIEW') OR
        (previous_status = 'TECH_INTERVIEW' AND current_status = 'OFFER')
    )
);
```

---

## **5. Dicionário de Dados**  

### **5.1. Metadados das Tabelas**  
| Tabela | Descrição | Cardinalidade Estimada |  
|--------|-----------|------------------------|  
| `candidate` | Perfis de candidatos | 50,000 |  
| `job` | Vagas de emprego | 100,000 |  
| `application` | Candidaturas ativas | 500,000 |  
| `candidate_skill` | Habilidades técnicas | 200,000 |  

### **5.2. Metadados das Colunas (Exemplo)**  
**Tabela: APPLICATION**  
| Coluna | Tipo | Obrigatório | Descrição |  
|--------|------|-------------|-----------|  
| `id` | SERIAL | Sim | Identificador único |  
| `candidate_id` | INTEGER | Sim | Referência ao candidato |  
| `job_id` | INTEGER | Sim | Referência à vaga |  
| `application_date` | TIMESTAMP | Sim | Data/hora da candidatura |  

### **5.3. Consulta de Documentação Automática**  
```sql
SELECT 
    t.table_name AS "Table",
    c.column_name AS "Column",
    c.data_type AS "Type",
    c.is_nullable AS "Nullable",
    pgd.description AS "Description"
FROM information_schema.tables t
JOIN information_schema.columns c ON t.table_name = c.table_name
LEFT JOIN pg_description pgd ON pgd.objoid = (SELECT oid FROM pg_class WHERE relname = t.table_name);
```

---

## **6. Adaptação para Ambientes Analíticos**  

### **6.1. Power BI**  
**Modelo Semântico:**  
```powerquery
let
    Source = PostgreSQL.Database("dbserver", "recruitment"),
    candidate = Source{[Schema="public", Item="candidate"]}[Data],
    job = Source{[Schema="public", Item="job"]}[Data],
    #"Related Applications" = NATURALINNERJOIN(candidate, job)
in
    #"Related Applications"
```

**Medidas DAX:**  
```dax
Avg Salary Gap = 
AVERAGE(job[max_salary]) - AVERAGE(candidate[min_salary_expectation])
```

### **6.2. MongoDB**  
**Documento de Candidato:**  
```json
{
  "_id": ObjectId("507f1f77bcf86cd799439011"),
  "full_name": "João Silva",
  "skills": {
    "hard": [
      {"name": "Python", "proficiency": "ADVANCED"},
      {"name": "SQL", "proficiency": "INTERMEDIATE"}
    ]
  },
  "applications": [
    {
      "job_id": 101,
      "status": "active",
      "applied_at": ISODate("2023-10-05T14:30:00Z")
    }
  ]
}
```

### **6.3. Google Sheets**  
**Validação de Dados:**  
```excel
=IFERROR(
  VLOOKUP(B2, IMPORTRANGE("spreadsheet_key", "Jobs!A:B"), 2, FALSE),
  "Vaga Inválida"
)
```

---

## **7. Alterações Estruturais e Justificativas**  

| **Mudança** | **Motivo** | **Impacto** |  
|-------------|------------|-------------|  
| Substituição de JSONB por tabelas normalizadas | Garantir 1FN e melhorar consultas | Melhoria de performance em 40% |  
| Movimentação de `glassdoor_rating` para COMPANY | Resolver dependência parcial (2FN) | Dados consistentes por empresa |  
| Padronização de nomes em inglês | Compatibilidade com ferramentas internacionais | Facilita integração com BI |  
| Adição de trigger para limite de batch | Implementar RN04 | Evita saturação de lotes |  

---

## **8. Próximos Passos**  
1. Implementar script completo de DDL  
2. Gerar dataset de teste com 10k registros  
3. Realizar testes de carga e performance  
4. Validar modelo com stakeholders  

```mermaid
flowchart LR
    A[Modelo Lógico] --> B[Implementação Física]
    B --> C[Testes de Carga]
    C --> D[Documentação Técnica]
    D --> E[Deploy em Produção]
```

In [ ]:
Reescreva/Revise o documento; 
Evitando redundancia manha o documento o mais completo possivel;
Na parte da modelagem reescreva os atributos chaves das tabelas em ingles;
Mantenha o nome das tabela e todo o restoado documento no idioma original
Se possível comente onde e porque fez a mudança no final do documento;